# Welcome to the Lab 🥼🧪

This one is only for the brave.

In [4]:
import os
import sys
import json
import subprocess
from datetime import datetime, timedelta
from urllib.request import urlopen

# Collab setup from one click above
if "google.colab" in sys.modules:
    from google.colab import userdata
    %pip install parcllabs plotly kaleido
    api_key = userdata.get('PARCL_LABS_API_KEY')
else:
    api_key = os.getenv('PARCL_LABS_API_KEY')

In [5]:
import parcllabs
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from parcllabs import ParclLabsClient


print(f"Parcl Labs Version: {parcllabs.__version__}")

ModuleNotFoundError: No module named 'parcllabs'

In [3]:
# Initialize the Parcl Labs client
client = ParclLabsClient(api_key)

In [4]:
# set nb config
pf_options = {
    'rental': 'rental_price_feed',
    'pricefeed': 'price_feed'
}

PF_TYPE = pf_options['rental']

In [5]:

labs_logo_lookup = {
    'blue': 'https://parcllabs-assets.s3.amazonaws.com/powered-by-parcllabs-api.png',
    'white': 'https://parcllabs-assets.s3.amazonaws.com/powered-by-parcllabs-api-logo-white+(1).svg'
}

# Set charting constants
labs_logo_dict = dict(
    source=labs_logo_lookup['white'],
    xref="paper",
    yref="paper",
    x=0.5,  # Centering the logo below the title
    y=1.04,  # Adjust this value to position the logo just below the title
    sizex=0.15, 
    sizey=0.15,
    xanchor="center",
    yanchor="bottom"
)

def build_chart(
        market_name: str, 
        data: pd.DataFrame,
        pf_type: str = 'price_feed',
    ):

    pf_type_format = {
        'price_feed': 'Price per Square Foot ($)',
        'rental_price_feed': 'Rental Price per Square Foot ($)',
    }

    HEIGHT = 900
    WIDTH = 1600
    
    # Calculate median price_feed
    median_price_feed = data[pf_type].median()
    
    fig = go.Figure()

    # Split data into continuous segments based on median
    segments = []
    current_segment = []
    current_color = None

    for i in range(len(data)):
        if current_color is None:
            current_color = '#FFFFFF' if data.iloc[i][pf_type] >= median_price_feed else '#57A3FF'
            current_segment.append(data.iloc[i])
        elif (data.iloc[i][pf_type] >= median_price_feed and current_color == '#FFFFFF') or (data.iloc[i][pf_type] < median_price_feed and current_color == '#57A3FF'):
            current_segment.append(data.iloc[i])
        else:
            segments.append((current_segment, current_color))
            current_color = '#FFFFFF' if data.iloc[i][pf_type] >= median_price_feed else '#57A3FF'
            current_segment = [data.iloc[i]]
    
    if current_segment:
        segments.append((current_segment, current_color))

    for segment, color in segments:
        segment_df = pd.DataFrame(segment)
        fig.add_trace(go.Scatter(
            x=segment_df['date'],
            y=segment_df[pf_type],
            mode='lines',
            line=dict(width=2, color=color),  # Reduced line width for thinner lines
            showlegend=False
        ))

    # Add horizontal line for median price feed value
    fig.add_shape(
        type="line",
        x0=data['date'].min(),
        y0=median_price_feed,
        x1=data['date'].max(),
        y1=median_price_feed,
        line=dict(
            color="#FFFFFF",
            width=1,
            dash="dot",  # Small dots for the median line
        ),
        opacity=1  # Set the opacity to 0.7
    )
    
    fig.add_layout_image(labs_logo_dict)
    
    fig.update_layout(
        margin=dict(l=0, r=0, t=110, b=0),
        height=HEIGHT,
        width=WIDTH,
        title={
            'text': f'{pf_type_format[pf_type]}: {market_name}',
            'y': 0.99,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(size=28, color='#FFFFFF'),
        },
        plot_bgcolor='#000000',  # Dark background for better contrast
        paper_bgcolor='#000000',  # Dark background for the paper
        font=dict(color='#FFFFFF'),
        xaxis=dict(
            title_text='',
            showgrid=False,  # Disable vertical grid lines
            tickangle=-45,
            tickfont=dict(size=14),
            linecolor='rgba(255, 255, 255, 0.7)',  # Axis line color with opacity
            linewidth=1  # Axis line width
        ),
        yaxis=dict(
            title_text=pf_type_format[pf_type],
            showgrid=True,
            gridwidth=0.5,  # Horizontal grid line width
            gridcolor='rgba(255, 255, 255, 0.2)',  # Horizontal grid line color with opacity
            tickfont=dict(size=14),
            tickprefix='$',  # Add dollar sign to y-axis labels
            zeroline=False,
            linecolor='rgba(255, 255, 255, 0.7)',  # Axis line color with opacity
            linewidth=1  # Axis line width
        ),
        hovermode='x unified',  # Unified hover mode for better interactivity
        hoverlabel=dict(
            bgcolor='#1F1F1F',
            font_size=14,
            font_family="Rockwell"
        )
    )

    root = f'../../graphics/{pf_type}'
    # timestamp
    timestamp = datetime.now().strftime('%Y-%m-%d')
    # join the path
    path = os.path.join(root, timestamp)
    if not os.path.exists(path):
        os.makedirs(path)

    # Save the plot
    fig.write_image(os.path.join(path, f'{market_name}_{pf_type}.png'), width=WIDTH, height=HEIGHT)
    
    # Show the plot
    fig.show()

def format_names(nme):
    if nme == 'United States Of America':
        return 'USA'
    state = nme.split(',')[-1].strip().upper().split('-')[0]
    metro = nme.split(',')[0].split('-')[0].strip()
    metro = metro.split('/')[0].strip()
    return f"{metro}, {state}"


def calculate_percent_changes(
        data, 
        raw: bool=False,
        pf_type: str = 'price_feed'
    ):
    # Ensure the date column is in datetime format
    data['date'] = pd.to_datetime(data['date'])
    
    # Sort the data by date
    data = data.sort_values(by='date').reset_index(drop=True)
    
    # Get the start and current price_feed values
    start_price = data.iloc[0][pf_type]
    start_date = data.iloc[0]['date']
    start_date_year = start_date.year
    current_price = data.iloc[-1][pf_type]
    
    # Helper function to get the price at a specific date
    def get_price_at_date(date):
        filtered_data = data[data['date'] <= date]
        if not filtered_data.empty:
            return filtered_data.iloc[-1][pf_type]
        else:
            return None
        
    # Calculate percent changes
    def percent_change(old, new, raw=raw):
        if old is not None and new is not None:
            change = ((new - old) / old)
            emoji = '📈' if change > 0 else '📉'
            if raw:
                return change
            else:
                change = change * 100
                return f"{change:.2f}% {emoji}"
        else:
            return None
        
    
    changes = {}
    # Define the date ranges
    now = data.iloc[-1]['date']
    one_year_ago = now - timedelta(days=365)
    six_months_ago = now - timedelta(days=6*30)
    thirty_days_ago = now - timedelta(days=30)

    price_1_year_ago = get_price_at_date(one_year_ago)
    price_6_months_ago = get_price_at_date(six_months_ago)
    price_30_days_ago = get_price_at_date(thirty_days_ago)

    changes.update({
        '% Change (30 Day)': percent_change(price_30_days_ago, current_price, raw=raw),
        '% Change (6 mo)': percent_change(price_6_months_ago, current_price, raw=raw),
        '% Change (YoY)': percent_change(price_1_year_ago, current_price, raw=raw),
    })

    if pf_type == 'price_feed':
        five_years_ago = now - timedelta(days=5*365)
        price_5_years_ago = get_price_at_date(five_years_ago)
        changes.update({'% Change (5 yr)': percent_change(price_5_years_ago, current_price, raw=raw)})
    else:
        four_years_ago = now - timedelta(days=4*365)
        price_4_years_ago = get_price_at_date(four_years_ago)
        changes.update({'% Change (4 yr)': percent_change(price_4_years_ago, current_price, raw=raw)})
        
    changes.update({f'% Change (Since `{str(start_date_year)[-2:]})': percent_change(start_price, current_price, raw=raw)})
    
    return changes


In [6]:
# lets get all US markets currently available to trade on the Parcl Exchange
# Now lets say you want all price feed markets that are on the parcl exchange
market_df = client.search_markets.retrieve(
    sort_by='PARCL_EXCHANGE_MARKET',
    # location_type='CBSA',
    sort_order='DESC',
    as_dataframe=True,
    params={'limit': 14},  # expand the default limit to 14, as of this writing, 14 markets are available
)

# add united states

# us = client.search_markets.retrieve(
#     query='United States',
#     sort_by='TOTAL_POPULATION',
#     sort_order='DESC',
#     as_dataframe=True,
#     params={'limit': 1},  # expand the default limit to 14, as of this writing, 14 markets are available
# )

# market_df = pd.concat([us, market_df], ignore_index=True)

# filter to pricefeed markets
market_df = market_df.loc[market_df['pricefeed_market']==1]
market_df['name'] = market_df['name'].apply(format_names)
parcl_ids = market_df['parcl_id'].tolist()
market_df.head(20)

,parcl_id,country,geoid,state_fips_code,name,state_abbreviation,region,location_type,total_population,median_income,parcl_exchange_market,pricefeed_market,case_shiller_10_market,case_shiller_20_market
0,5384169,USA,1304000,13,"Atlanta City, ATLANTA CITY",GA,SOUTH_ATLANTIC,CITY,494838,77655,1,1,0,0
1,5407714,USA,2507000,25,"Boston City, BOSTON CITY",MA,NEW_ENGLAND,CITY,665945,89212,1,1,0,0
2,5387853,USA,1714000,17,"Chicago City, CHICAGO CITY",IL,EAST_NORTH_CENTRAL,CITY,2721914,71673,1,1,0,0
3,5380879,USA,4805000,48,"Austin City, AUSTIN CITY",TX,WEST_SOUTH_CENTRAL,CITY,958202,86556,1,1,0,0
4,5353022,USA,1245025,12,"Miami Beach City, MIAMI BEACH CITY",FL,SOUTH_ATLANTIC,CITY,82400,65116,1,1,0,0
5,5377230,USA,3240000,32,"Las Vegas City, LAS VEGAS CITY",NV,MOUNTAIN,CITY,644835,66356,1,1,0,0
6,5503877,USA,1150000,11,"Washington City, WASHINGTON CITY",DC,SOUTH_ATLANTIC,CITY,670587,101722,1,1,0,0
7,5822447,USA,36047,36,"Kings County, KINGS COUNTY",NY,MIDDLE_ATLANTIC,COUNTY,2679620,74692,1,1,0,0
8,5372594,USA,3651000,36,"New York City, NEW YORK CITY",NY,MIDDLE_ATLANTIC,CITY,8622467,76607,1,1,0,0
9,5373892,USA,0644000,06,"Los Angeles City, LOS ANGELES CITY",CA,PACIFIC,CITY,3881041,76244,1,1,0,0


In [7]:
# lets retrieve data back to 2011 for these price feeds
if PF_TYPE == 'price_feed':
    START_DATE = '2010-01-01'
    feeds = client.price_feed.retrieve_many(
        parcl_ids=parcl_ids,
        start_date=START_DATE,
        as_dataframe=True,
        params={'limit': 1000},  # expand the limit to 1000, these are daily series
        auto_paginate=True, # auto paginate to get all the data - WARNING: ~6k credits can be used in one parcl price feed. Change the START_DATE to a more recent date to reduce the number of credits used
    )
if PF_TYPE == 'rental_price_feed':
    START_DATE = '2020-01-01'
    feeds = client.rental_price_feed.retrieve_many(
        parcl_ids=parcl_ids,
        start_date=START_DATE,
        as_dataframe=True,
        params={'limit': 1000},  # expand the limit to 1000, these are daily series
        auto_paginate=True, # auto paginate to get all the data - WARNING: ~6k credits can be used in one parcl price feed. Change the START_DATE to a more recent date to reduce the number of credits used
    )



feeds.head()

|█████████████████████████████████████▏⚠︎ | (!) 13/14 [93%] in 5.3s (2.44/s) 


,date,rental_price_feed,parcl_id
0,2024-06-01,1.719,5384169
1,2024-05-31,1.715,5384169
2,2024-05-30,1.709,5384169
3,2024-05-29,1.704,5384169
4,2024-05-28,1.700,5384169


In [8]:
feeds = feeds.merge(market_df[['parcl_id', 'name']], on='parcl_id', how='left')
feeds

,date,rental_price_feed,parcl_id,name
0,2024-06-01,1.719,5384169,"Atlanta City, ATLANTA CITY"
1,2024-05-31,1.715,5384169,"Atlanta City, ATLANTA CITY"
2,2024-05-30,1.709,5384169,"Atlanta City, ATLANTA CITY"
3,2024-05-29,1.704,5384169,"Atlanta City, ATLANTA CITY"
4,2024-05-28,1.700,5384169,"Atlanta City, ATLANTA CITY"
...,...,...,...,...
20977,2020-01-05,2.414,5374167,"San Diego City, SAN DIEGO CITY"
20978,2020-01-04,2.413,5374167,"San Diego City, SAN DIEGO CITY"
20979,2020-01-03,2.413,5374167,"San Diego City, SAN DIEGO CITY"
20980,2020-01-02,2.415,5374167,"San Diego City, SAN DIEGO CITY"


In [9]:
all_data = []

feeds = feeds.sort_values('name')
names = feeds['name'].unique()
total_markets = len(names)

for idx, pid in enumerate(feeds.sort_values('name')['parcl_id'].unique()):
    data = feeds.loc[feeds['parcl_id'] == pid].sort_values('date')
    name = data['name'].iloc[0].replace('Kings County', 'Brooklyn County').replace('Washington City', 'Washington, DC')
    build_chart(name, data, pf_type=PF_TYPE)
    changes = calculate_percent_changes(data, raw=False, pf_type=PF_TYPE)
    changes_raw = calculate_percent_changes(data, raw=True, pf_type=PF_TYPE)
    print(name + '\n')
    for k, v in changes.items():
        print(f"{k}: {v}")
    print('\n')
    if idx < total_markets - 1:
        print(f'Up next: {names[idx+1]}')
    # print('Trade today on: @parcl')
    # create row
    row = pd.DataFrame(changes_raw, index=[0])
    row['name'] = name
    all_data.append(row)

Atlanta City, ATLANTA CITY

% Change (30 Day): 4.31% 📈
% Change (6 mo): 4.82% 📈
% Change (YoY): 10.05% 📈
% Change (4 yr): 10.26% 📈
% Change (Since `20): 16.94% 📈


Up next: Austin City, AUSTIN CITY


Austin City, AUSTIN CITY

% Change (30 Day): -1.39% 📉
% Change (6 mo): 3.97% 📈
% Change (YoY): -9.00% 📉
% Change (4 yr): 6.98% 📈
% Change (Since `20): 16.27% 📈


Up next: Boston City, BOSTON CITY


Boston City, BOSTON CITY

% Change (30 Day): 0.43% 📈
% Change (6 mo): 3.26% 📈
% Change (YoY): -40.26% 📉
% Change (4 yr): 3.87% 📈
% Change (Since `20): 8.27% 📈


Up next: Chicago City, CHICAGO CITY


Chicago City, CHICAGO CITY

% Change (30 Day): -0.60% 📉
% Change (6 mo): 8.42% 📈
% Change (YoY): -8.95% 📉
% Change (4 yr): 10.48% 📈
% Change (Since `20): 20.16% 📈


Up next: Denver City, DENVER CITY


Denver City, DENVER CITY

% Change (30 Day): -2.84% 📉
% Change (6 mo): 3.59% 📈
% Change (YoY): 1.10% 📈
% Change (4 yr): 16.64% 📈
% Change (Since `20): 25.40% 📈


Up next: Kings County, KINGS COUNTY


...

Miami Beach City, MIAMI BEACH CITY

% Change (30 Day): -0.54% 📉
% Change (6 mo): 3.81% 📈
% Change (YoY): -7.59% 📉
% Change (4 yr): 44.54% 📈
% Change (Since `20): 51.44% 📈


Up next: New York City, NEW YORK CITY


New York City, NEW YORK CITY

% Change (30 Day): 2.38% 📈
% Change (6 mo): -2.47% 📉
% Change (YoY): -43.79% 📉
% Change (4 yr): 3.49% 📈
% Change (Since `20): 10.08% 📈


Up next: San Diego City, SAN DIEGO CITY


San Diego City, SAN DIEGO CITY

% Change (30 Day): 0.25% 📈
% Change (6 mo): -0.87% 📉
% Change (YoY): 0.47% 📈
% Change (4 yr): 31.57% 📈
% Change (Since `20): 31.78% 📈


Up next: San Francisco City, SAN FRANCISCO CITY


San Francisco City, SAN FRANCISCO CITY

% Change (30 Day): 2.37% 📈
% Change (6 mo): -1.75% 📉
% Change (YoY): 2.05% 📈
% Change (4 yr): -10.24% 📉
% Change (Since `20): -11.71% 📉


Up next: USA


USA

% Change (30 Day): -0.61% 📉
% Change (6 mo): 2.51% 📈
% Change (YoY): -1.14% 📉
% Change (4 yr): 6.14% 📈
% Change (Since `20): 28.50% 📈




In [ ]:
output = pd.concat(all_data)
output.head()

output.shape

In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Use your actual data here
if PF_TYPE == 'price_feed':
    df = output.sort_values('% Change (Since `10)', ascending=False).rename(columns={'volatility': "Annualized Volatility"})
else:
    df = output.sort_values('% Change (Since `20)', ascending=False)
df['name'] = df['name'].replace({'United States Of America': 'USA'})

column_headers = [col for col in df.columns if col != 'name']

pf_type_format = {
        'price_feed': 'Price per Square Foot ($)',
        'rental_price_feed': 'Rental Price per Square Foot ($)',
    }

# Function to format the percentage with arrows
def format_percent(value, show_arrow=True):
    formatted_value = f"{value:6.2%}"  # Fixed width of 6 characters
    if value > 0 and show_arrow:
        return f"<b>{formatted_value} ⬆️</b>"  # Up arrow
    elif value < 0 and show_arrow:
        return f"<b>{formatted_value} ⬇️</b>"  # Down arrow
    else:
        return f"<b>{formatted_value}</b>"

# Define the function to scale the color based on the value
def color_scale(value, min_val, max_val):
    if value < 0:
        r = 255
        g = 0
        b = 0
    else:
        normalized = (value - min_val) / (max_val - min_val)
        r = 0
        g = int(100 + 155 * normalized)  # Starts from 100 to ensure readability
        b = 0
    return f'rgb({r},{g},{b})'

# Prepare data and colors for the table
colors = [[] for _ in range(len(df.columns))]
formatted_data = [[] for _ in range(len(df.columns))]

# Calculate color scales for each column
for col in df.columns:
    if col != 'name' and col != 'Annualized Volatility':
        min_val = df[col].min()
        max_val = df[col].max()
        for value in df[col]:
            formatted_data[df.columns.get_loc(col)].append(format_percent(value))
            colors[df.columns.get_loc(col)].append(color_scale(value, min_val, max_val))
    else:
        for value in df[col]:
            if col == 'name':
                formatted_data[df.columns.get_loc(col)].append(f"<b>{value}</b>")
                colors[df.columns.get_loc(col)].append('#000000')  # Black for name column
            else:
                formatted_data[df.columns.get_loc(col)].append(f"<b>{value:.2%}</b>")
                colors[df.columns.get_loc(col)].append('#000000')  # Black for volatility column

# Define headers and table layout
fig = go.Figure(data=[go.Table(
    header=dict(values=['<b>Market</b>'] + [f"<b>{header}</b>" for header in column_headers],
                fill_color='#000000',  # Black for header
                font=dict(color='#FFFFFF', size=12),
                align='center',
                height=30),
    cells=dict(values=[formatted_data[df.columns.get_loc('name')]] + 
               [formatted_data[df.columns.get_loc(col)] for col in column_headers],
               fill=dict(color=[['#000000']*len(df)] + colors),
               font=dict(color='#FFFFFF', size=12),
               align='center',
               height=30)
)])

# Add the logo image
labs_logo_lookup = {
    'blue': 'https://parcllabs-assets.s3.amazonaws.com/powered-by-parcllabs-api.png',
    'white': 'https://parcllabs-assets.s3.amazonaws.com/powered-by-parcllabs-api-logo-white+(1).svg'
}
labs_logo_dict = dict(
    source=labs_logo_lookup['white'],
    xref="paper",
    yref="paper",
    x=0.5,
    y=1.01,
    sizex=0.2,
    sizey=0.2,
    xanchor="center",
    yanchor="bottom"
)
fig.add_layout_image(labs_logo_dict)

w = 1100
h = 1600

# Update layout and display the figure
type_text = 'Rental' if PF_TYPE == 'rental_price_feed' else ''
fig.update_layout(
    title={
        'text': f"{type_text} Price Feed Market Comparison",
        'y': 0.97,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    title_font_color='#FFFFFF',
    title_font_size=24,
    width=w,  # Increase the width for wider cells
    height=h,
    paper_bgcolor='#080D16',
    margin=dict(l=10, r=10, t=120, b=10)
)

root = f'../../graphics/{PF_TYPE}'
# timestamp
timestamp = datetime.now().strftime('%Y-%m-%d')
# join the path
path = os.path.join(root, timestamp)
if not os.path.exists(path):
    os.makedirs(path)

# Save the plot
fig.write_image(os.path.join(path, f'comp_table_{PF_TYPE}.png'), width=w, height=h)

fig.show()

